<h1>Learning Portfolio Artifact 5</h1>

The paper "Scheduling a Bridge Club (A Case Study in Discrete Optimization)" by Bruce S. Elenbogen and Bruce R. Maxim explores the complexities of scheduling for a bridge club, presenting various methods of discrete optimization to tackle the problem. 

These are the detailed steps and methods described in the paper "Scheduling a Bridge Club: A Case Study in Discrete Optimization" by Bruce S. Elenbogen and Bruce R. Maxim. The authors explore various techniques to solve a scheduling problem presented by a local bridge club, aiming to find an optimal schedule for the club's meetings.

1. **Introduction:** The paper introduces the scheduling problem presented by the bridge club and highlights the complexity involved despite its seemingly simple nature. It discusses the significance of discrete optimization techniques in solving such problems efficiently.

2. **Problem Definition:** The bridge club consists of 12 couples divided into 3 groups of 4 couples each for each meeting. The goal is to schedule 8 meetings a year, ensuring that each couple competes against other couples an equal number of times. If equal matchups are not possible, the objective is to minimize the discrepancy in the number of matchups.

3. **Decision Variables:** Assigning each couple to one of the three groups (A, B, or C) for each of the 8 meetings throughout the year.
4. **Constraints:**
    - Each couple must be assigned to a group for each meeting.
    - Each group must consist of exactly 4 couples.
    - Each couple competes against the other 11 couples a certain number of times, which should be balanced across all couples as much as possible.
    - The number of times any couple competes against any opposing couple should be the same for all couples, or as nearly the same as possible.
    - The total number of games played per meeting should be 18, with each couple participating in 6 games.
    - The total number of games played throughout the year should be 144.<br><br>


4. **Optimal Solutions:** The authors define the problem's optimality using a competition matrix and propose a cost function to measure the quality of a schedule. They discuss the classification of the problem as a resolvable partially balanced incomplete block design (BIBD).

5. **Methods of Discrete Optimization:**
   - **Method 1 (Exhaustive Search):** The authors discuss the impracticality of exhaustively searching all possible schedules due to the vast number of permutations.
   - **Method 2 (Greedy Algorithm):** They propose a greedy algorithm that minimizes the total number of repeated pairings of teams at each meeting. This method is computationally feasible and yields a suboptimal solution.
   - **Method 3 (Branch and Bound):** The authors explore a branch and bound approach, but due to the large search space, it proves to be impractical.
   - **Method 4 (Steepest Descent):** This method involves iteratively searching for neighboring schedules with lower costs. It's computationally efficient but heavily dependent on the initial schedule.
   - **Method 5 (Annealed Search):** An annealed search method inspired by simulated annealing is discussed, where neighboring schedules are considered based on their costs relative to the current schedule. This method aims to avoid getting stuck in local optima.<br><br>

6. **Results:** The paper presents the results obtained from each method, including the quality of schedules produced and the computational resources required.

Overall, the paper provides a comprehensive exploration of discrete optimization techniques applied to a real-world scheduling problem, highlighting the challenges and trade-offs involved in finding optimal solutions.

#### So, let's try to solve a problem using the PuLP<br>

To solve the scheduling problem using the PuLP library in Python, we need to define the decision variables, constraints, and the objective function. <br>


In [35]:
import random
from pulp import *

Let's create a scheduling problem for a small team with different tasks and constraints. 

Problem Description:
- There are 4 team members: Alice, Bob, Charlie, and David.
- There are 5 tasks to be completed: Task 1, Task 2, Task 3, Task 4, and Task 5.
- Each task has a duration and a deadline.
- Each team member has specific skills and availability.

We want to assign tasks to team members in a way that maximizes the total skill level of the assigned team members while ensuring that each task is completed before its deadline and that team members don't have conflicting assignments.

Let's define the problem more concretely:

- Task 1: Duration = 2 hours, Deadline = 10 hours
- Task 2: Duration = 3 hours, Deadline = 12 hours
- Task 3: Duration = 1 hour, Deadline = 9 hours
- Task 4: Duration = 4 hours, Deadline = 15 hours
- Task 5: Duration = 2 hours, Deadline = 11 hours

- Skills and availability of team members:
  - Alice: Skill Level = 8, Available for 6 hours
  - Bob: Skill Level = 7, Available for 8 hours
  - Charlie: Skill Level = 6, Available for 7 hours
  - David: Skill Level = 5, Available for 5 hours


**Objective Function**:
The objective function aims to maximize the total skill level of the assigned team members. It is formulated as follows:

$$
\text{Maximize} \quad Z = \sum_{\text{task } i} \sum_{\text{team member } j} \text{skill}_{ij} \times \text{assign}_{ij}
$$

Where:
- {skill}_{ij} is the skill level of team member j for task i.
- {assign}_{ij} is a binary variable indicating whether task i is assigned to team member j.

**Decision Variables**:
For each task \( i \) and each team member \( j \), we define a binary decision variable \( \text{assign}_{ij} \) that takes the value 1 if task \( i \) is assigned to team member \( j \), and 0 otherwise.

**Constraints**:
1. **Task Assignment Constraints**: Each task must be assigned to exactly one team member.
$$
\sum_{\text{team member } j} \text{assign}_{ij} = 1 \quad \text{for each task } i
$$
2. **Deadline Constraints**: The total duration of tasks assigned to a team member should not exceed their availability for each team member \( j \).
$$
\sum_{\text{task } i} \text{duration}_i \times \text{assign}_{ij} \leq \text{availability}_j \quad \text{for each team member } j
$$
3. **Deadline Constraints**: Each task must be completed before its deadline.
$$
\sum_{\text{team member } j} \text{duration}_i \times \text{assign}_{ij} \leq \text{deadline}_i \quad \text{for each task } i
$$


In [36]:

# Define tasks
tasks = ['Task1', 'Task2', 'Task3', 'Task4', 'Task5']

# Task durations and deadlines
durations = {'Task1': 2, 'Task2': 3, 'Task3': 1, 'Task4': 4, 'Task5': 2}
deadlines = {'Task1': 10, 'Task2': 12, 'Task3': 9, 'Task4': 15, 'Task5': 11}

# Define team members
team_members = ['Alice', 'Bob', 'Charlie', 'David']

# Skills and availability of team members
skills = {'Alice': 8, 'Bob': 7, 'Charlie': 6, 'David': 5}
availability = {'Alice': 6, 'Bob': 8, 'Charlie': 7, 'David': 5}

# Create a LP maximization problem
prob = LpProblem("Scheduling Problem", LpMaximize)

# Define variables for assignment of tasks to team members
assign = LpVariable.dicts("Assign", (tasks, team_members), cat='Binary')

# Objective function: maximize total skill level
prob += lpSum([skills[member] * assign[task][member] for task in tasks for member in team_members])

# Constraints: task completion before deadline and availability of team members
for task in tasks:
    prob += lpSum([assign[task][member] for member in team_members]) == 1  # Each task is assigned to exactly one member
    prob += lpSum([assign[task][member] * durations[task] for member in team_members]) <= deadlines[task]
for member in team_members:
    prob += lpSum([assign[task][member] * durations[task] for task in tasks]) <= availability[member]

# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/abheetkansal/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/wx/94p7x06n6s722d_3zpw_tfhh0000gn/T/88fddce51328413b98ef995d4941d3a2-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/wx/94p7x06n6s722d_3zpw_tfhh0000gn/T/88fddce51328413b98ef995d4941d3a2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 19 COLUMNS
At line 140 RHS
At line 155 BOUNDS
At line 176 ENDATA
Problem MODEL has 14 rows, 20 columns and 60 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 38.3333 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 10 rows, 20 columns (20 integer (20 of which binary)) and 46 ele

1

In [37]:

print("Status:", LpStatus[prob.status])
print("Optimal Assignment:")
for task in tasks:
    for member in team_members:
        if assign[task][member].varValue == 1:
            print(f"Assign {task} to {member}")
print("Total Skill Level:", value(prob.objective))


Status: Optimal
Optimal Assignment:
Assign Task1 to Alice
Assign Task2 to Bob
Assign Task3 to Alice
Assign Task4 to Bob
Assign Task5 to Alice
Total Skill Level: 38.0
